In [1]:
# net level
import pNN_NetLevel_LNC_variation as pNN

# packages
import os
import math
import torch
import random
import pickle
import calendar
import numpy as np
import config
import training_together_variation as training 
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
datasets = os.listdir('./dataset')
datasets = [f for f in datasets if (f.startswith('Dataset') and f.endswith('.p'))] 
#datasets

In [3]:

#result = torch.zeros((13, 1, 10))
lrs = [0]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
        
        for n in range(10):     
            
        setup = f'dataset_{data_name}_seed_{seed}_nonlearnable1'
        print(setup)
            
        if os.path.exists(f'./results_variation/pNN_{setup}'):
            print('File exists, pass.')
        else:
            torch.manual_seed(seed)
            model = pNN.pNN([N_feature, 3, N_class], 1, 0)
    
            optimizer = torch.optim.Adam([{'params':model.GetParam('theta_'),'lr':1e-1},
                                  {'params':model.GetParam('eta_'), 'lr':lr},
                                  {'params':model.GetParam('inv_'),'lr':lr}])
    
    
            NN, acc_valid = training.training_pNN(model, train_loader, valid_loader, optimizer, pNN.lossfunction)
                #result[i,j,n] = acc_valid
            torch.save(NN, f'./results_variation/pNN_{setup}')

dataset_acuteinflammation_seed_0_nonlearnable1
| Epoch:        0 | Valid accuracy: 0.86957 | Valid loss: 0.676321149 |
| Epoch:     5000 | Valid accuracy: 1.00000 | Valid loss: 0.000000000 |
dataset_balancescale_seed_0_nonlearnable1
| Epoch:        0 | Valid accuracy: 0.41935 | Valid loss: 0.713952243 |
| Epoch:     5000 | Valid accuracy: 0.96774 | Valid loss: 0.048945166 |
dataset_breastcancerwisc_seed_0_nonlearnable1
| Epoch:        0 | Valid accuracy: 0.66187 | Valid loss: 0.698530138 |
| Epoch:     5000 | Valid accuracy: 0.94964 | Valid loss: 0.090649277 |
dataset_cardiotocography3clases_seed_0_nonlearnable1
| Epoch:        0 | Valid accuracy: 0.74528 | Valid loss: 0.702790558 |
| Epoch:     5000 | Valid accuracy: 0.90566 | Valid loss: 0.169152781 |
dataset_energyy1_seed_0_nonlearnable1
| Epoch:        0 | Valid accuracy: 0.20261 | Valid loss: 0.725977361 |
| Epoch:     5000 | Valid accuracy: 0.84314 | Valid loss: 0.240390152 |
| Epoch:    10000 | Valid accuracy: 0.87582 | Valid lo

## valid_acc

In [6]:

acc = torch.zeros((13, 1, 10))
#lrs = [0.0005]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
        
        for n in range(10):
            seed = n      
            setup = f'dataset_{data_name}_seed_{seed}_nonlearnable1'
            model = torch.load(f'./results_variation/pNN_{setup}')
            
            
            with torch.no_grad():
                for X_valid, y_valid in valid_loader:
                
                    prediction_valid = model(X_valid)
                    #acc_valid = lossfunction(prediction_valid, y_valid)
                    yhat_valid = torch.argmax(prediction_valid.data, 2)
                
                    yy_valid = y_valid.repeat(prediction_valid.shape[0], 1)
                    valid_correct = torch.sum(yhat_valid == yy_valid.data)
                    acc_valid = valid_correct / (y_valid.numel() * prediction_valid.shape[0])
                    acc[i,j,n] = acc_valid
                    